In [ ]:
!pip install requests

API Docs: https://developer.twitter.com/en/docs/twitter-api/tweets/search/api-reference/get-tweets-search-recent

Example using Python: https://github.com/twitterdev/Twitter-API-v2-sample-code/blob/main/Recent-Search/recent_search.py



In [1]:
import os
import json
from typing import Callable

import requests

In [27]:
import pandas as pd

In [2]:
SEARCH_URL = "https://api.twitter.com/2/tweets/search/recent"
BEARER_TOKEN = "<bearer token>"

## Auth

In [4]:
def bearer_oauth(r):
    """Method required by bearer token authentication."""

    r.headers["Authorization"] = f"Bearer {BEARER_TOKEN}"
    r.headers["User-Agent"] = "v2RecentSearchPython"
    return r

## Search recent tweets

In [18]:
def build_params() -> dict:
    return {
        "query": "(atardecer OR sunset OR #sunset) has:images",
        "media.fields": "type,url,media_key",
        "expansions": "attachments.media_keys",
    }

In [6]:
def connect_to_endpoint(url: str, params: dict, auth_fn: Callable):
    response = requests.get(url, auth=auth_fn, params=params)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

In [ ]:
response = connect_to_endpoint(
    url=SEARCH_URL, params=build_params(), auth_fn=bearer_oauth
)

In [ ]:
print(json.dumps(response, indent=4, sort_keys=True))

In [31]:
def extract_data_from_response(raw_data: dict) -> pd.DataFrame:
    data = list()

    for item in raw_data["includes"]["media"]:
        if item["type"] != "photo":
            continue

        data.append([item["media_key"], item["type"], item["url"]])

    return pd.DataFrame(data, columns=["media_key", "type", "url"])

In [32]:
df = extract_data_from_response(response)

In [33]:
df

,media_key,type,url
0,3_1569841358705131523,photo,https://pbs.twimg.com/media/FckyZftWAAMCP8t.jpg
1,3_1568602867203809281,photo,https://pbs.twimg.com/media/FcTL_zVXoAEIvAT.png
2,3_1569651228770205696,photo,https://pbs.twimg.com/media/FciFeegagAABt9-.jpg
3,3_1569841617770536960,photo,https://pbs.twimg.com/media/FckyokzWYAAHF3H.jpg
4,3_1569841612162752512,photo,https://pbs.twimg.com/media/FckyoP6WYAAE5fC.jpg
5,3_1569749465136775169,photo,https://pbs.twimg.com/media/Fcje0lfWQAELndG.jpg
6,3_1569841568072220673,photo,https://pbs.twimg.com/media/FckylrqWQAEzyhO.jpg


In [35]:
for idx, row in df.iterrows():
    r = requests.get(row["url"], stream=True)

    if r.status_code == 200:
        with open(f"example_{idx}.png", "wb") as f:
            f.write(r.content)